### Using OpenAIs native function calling
In this notebook we will be exploring the use of OpenAI python sdk native function calling capabilities to handing function calling with language models

In [12]:
import os
import openai
from dotenv import load_dotenv

In [13]:
load_dotenv()

True

In [14]:
import json

# in this section we will be creating a hard coded weather function to return a static weather object

def get_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [15]:
get_weather("London")

'{"location": "London", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [16]:
functions = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string",
                         "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [17]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

In [20]:
client = openai.OpenAI(base_url="https://api.openai.com/v1")

In [22]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)

In [24]:
response.choices

[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_weather'), tool_calls=None))]

In [25]:
len(response.choices)

1

In [36]:
args = json.loads(response.choices[0].message.function_call.arguments)
name = response.choices[0].message.function_call.name

In [37]:
func = globals()[name]

In [39]:
func(**args)

'{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [40]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [41]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [42]:
response.choices[0].message.content

'Hello! How can I assist you today?'

In [44]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response) # has both function call and tools call

ChatCompletion(id='chatcmpl-8mqP38wE5blAyawICbP8tItNbDhYb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1706651873, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=75, total_tokens=85))


In [45]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

ChatCompletion(id='chatcmpl-8mqPtOt9jGKJTvb5hLezJ6BRyg7fE', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1706651925, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=76, total_tokens=85))


In [47]:
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

ChatCompletion(id='chatcmpl-8mqQhxkmWYoOJYOlU04gjzCm6VNDz', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Let me check the current weather in Boston for you.', role='assistant', function_call=None, tool_calls=None))], created=1706651975, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=81, total_tokens=92))


In [49]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-8mqSD5Fvf6MlV1kNcIm92redVTjPB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_weather'), tool_calls=None))], created=1706652069, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=87, total_tokens=98))


In [51]:
observation = '{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [52]:
messages.append(
        {
            "role": "function",
            "name": "get_current_weather",
            "content": observation,
        }
)

In [53]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
print(response)

ChatCompletion(id='chatcmpl-8mqTt1kivGezPijPYLzNH7uwIaSXt', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in Boston is 72°F and sunny. It is also windy.', role='assistant', function_call=None, tool_calls=None))], created=1706652173, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=53, total_tokens=70))
